In [1]:
import pandas as pd       
import matplotlib as mat
import matplotlib.pyplot as plt    
import numpy as np
import seaborn as sns

import random
import os
from numpy.random import seed
seed(42)
random.seed(42)

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import glob
import cv2

from tensorflow.random import set_seed


In [2]:
main_path="C:/Users/cool/Desktop/texniti noimosini/spl"

train_path = os.path.join(main_path,"train")
test_path=os.path.join(main_path,"test")


train_normal = glob.glob(train_path+"/NORMAL/*.jpeg")
train_pneumonia = glob.glob(train_path+"/BACTERIA/*.jpeg")
train_VIRUS = glob.glob(train_path+"/VIRUS/*.jpeg")

test_normal = glob.glob(test_path+"/NORMAL/*.jpeg")
test_pneumonia = glob.glob(test_path+"/BACTERIA/*.jpeg")
test_VIRUS = glob.glob(test_path+"/VIRUS/*.jpeg")


In [3]:
train_list = [x for x in train_normal]
train_list.extend([x for x in train_pneumonia])
train_list.extend([x for x in train_VIRUS])

df_train = pd.DataFrame(np.concatenate([['Normal']*len(train_normal) , ['Virus']*len(train_VIRUS) , ['Pneumonia']*len(train_pneumonia)]), columns = ['class'])
df_train['image'] = [x for x in train_list]

test_list = [x for x in test_normal]
test_list.extend([x for x in test_pneumonia])
test_list.extend([x for x in test_VIRUS])

df_test = pd.DataFrame(np.concatenate([['Normal']*len(test_normal) , ['Pneumonia']*len(test_pneumonia), ['Virus']*len(test_VIRUS)]), columns = ['class'])
df_test['image'] = [x for x in test_list]

In [4]:
print(df_test)
print(df_train)

      class                                              image
0    Normal  C:/Users/cool/Desktop/texniti noimosini/spl\te...
1    Normal  C:/Users/cool/Desktop/texniti noimosini/spl\te...
2    Normal  C:/Users/cool/Desktop/texniti noimosini/spl\te...
3    Normal  C:/Users/cool/Desktop/texniti noimosini/spl\te...
4    Normal  C:/Users/cool/Desktop/texniti noimosini/spl\te...
..      ...                                                ...
619   Virus  C:/Users/cool/Desktop/texniti noimosini/spl\te...
620   Virus  C:/Users/cool/Desktop/texniti noimosini/spl\te...
621   Virus  C:/Users/cool/Desktop/texniti noimosini/spl\te...
622   Virus  C:/Users/cool/Desktop/texniti noimosini/spl\te...
623   Virus  C:/Users/cool/Desktop/texniti noimosini/spl\te...

[624 rows x 2 columns]
          class                                              image
0        Normal  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
1        Normal  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
2        Normal  C:

In [5]:
IMG_SIZE = 224
BATCH = 32
SEED = 42

In [6]:
train_df, val_df = train_test_split(df_train, test_size = 0.20, shuffle=True, random_state = SEED, stratify = df_train['class'])
print(train_df.shape)
print(val_df[:20])

(4169, 2)
          class                                              image
3978  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
648      Normal  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
3267  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
4736  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
1263     Normal  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
4838  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
1275     Normal  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
1109     Normal  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
3113  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
3488  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
4276  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
3645  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
4929  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
1405      Virus  C:/Users/cool/Desktop/texniti noimo

In [7]:
# https://vijayabhaskar96.medium.com/tutorial-on-keras-flow-from-dataframe-1fd4493d237c

train_datagen = ImageDataGenerator(rescale=1/255.,
                                  zoom_range = 0.1,
                                  rotation_range = 0.1,
                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1)

val_datagen = ImageDataGenerator(rescale=1/255.)

ds_train = train_datagen.flow_from_dataframe(train_df,
                                             x_col = 'image',
                                             y_col = 'class',
                                             target_size = (IMG_SIZE, IMG_SIZE),
                                             class_mode = 'categorical',
                                             batch_size = BATCH,
                                             seed = SEED)

ds_val = val_datagen.flow_from_dataframe(val_df,                                            
                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            class_mode = 'categorical',
                                            batch_size = BATCH,
                                            seed = SEED)

ds_test = val_datagen.flow_from_dataframe(df_test,
                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            class_mode = 'categorical',
                                            batch_size = 1,
                                            shuffle = False)

Found 4169 validated image filenames belonging to 3 classes.
Found 1043 validated image filenames belonging to 3 classes.
Found 624 validated image filenames belonging to 3 classes.


In [8]:
print(train_df.shape)

(4169, 2)


In [9]:
#Setting callbakcs

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min',
)


In [10]:
from tensorflow.keras.applications.resnet50 import ResNet50



In [11]:
resnet1 = tf.keras.applications.resnet50.ResNet50(input_shape=(224, 224, 3),  include_top=True, weights='imagenet')

In [12]:
resnet1.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [13]:
# Επιλέγουμε να εκπαιδεύουμε όλα τα βάρη
resnet1.trainable = True
# Επιλέγουμε την έξοδο του τελευταίου συνελικτικού επιπέδου. 
# Θα μπορούσαμε να επιλέξουμε οποιοδήποτε άλλο επίπεδο.
conv_layer = resnet1.get_layer('conv5_block3_out').output
# Όπως προτείνεται και στο ResNet paper, επιλέγουμε, για κάθε εικόνα εξόδου 
# να πάρουμε το μέσο όρο (και όχι να χρησιμοποιήσουμε τη Flatten())
x = tf.keras.layers.GlobalAveragePooling2D()(conv_layer)
# Προσθέτουμε τα ταλευταία επίπεδα κατηγοριοποίησης
x = tf.keras.layers.Dense(3, activation='softmax')(x)
# Ορίζουμε το ενδιάμεσο μοντέλο
model1 = tf.keras.Model(resnet1.input, [x])

# Και ορίζουμε το συνολικό μοντέλο, όπως προηγουμένως
input = tf.keras.layers.Input((224,224,3))
pre = tf.keras.layers.Resizing(224, 224)(input)
out = model1(pre)
model2 = tf.keras.Model(input, outputs=[out])

In [14]:
model2.compile(loss='categorical_crossentropy'
              , optimizer = keras.optimizers.Adam(),metrics=["accuracy"])

model2.fit(ds_train, validation_data=ds_val,  batch_size=64, epochs=10)

score = model2.evaluate(ds_test)
print(score)


Epoch 1/10
131/131 [==============================] - 1475s 11s/step - loss: 0.7310 - accuracy: 0.6663 - val_loss: 1.8340 - val_accuracy: 0.4851
Epoch 2/10
131/131 [==============================] - 1356s 10s/step - loss: 0.5879 - accuracy: 0.6959 - val_loss: 1.7458 - val_accuracy: 0.4851
Epoch 3/10
131/131 [==============================] - 1286s 10s/step - loss: 0.5476 - accuracy: 0.7220 - val_loss: 2.6850 - val_accuracy: 0.2570
Epoch 4/10
131/131 [==============================] - 1431s 11s/step - loss: 0.5243 - accuracy: 0.7383 - val_loss: 2.5388 - val_accuracy: 0.4851
Epoch 5/10
131/131 [==============================] - 2744s 21s/step - loss: 0.5226 - accuracy: 0.7314 - val_loss: 3.3489 - val_accuracy: 0.2570
Epoch 6/10
131/131 [==============================] - 1991s 15s/step - loss: 0.5093 - accuracy: 0.7414 - val_loss: 2.5750 - val_accuracy: 0.4698
Epoch 7/10
131/131 [==============================] - 2221s 17s/step - loss: 0.4981 - accuracy: 0.7568 - val_loss: 0.7346 - val_ac

In [15]:
resnetf = tf.keras.applications.resnet50.ResNet50(input_shape=(224, 224, 3),  include_top=True, weights='imagenet')

resnetf.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [16]:
# Επιλέγουμε να εκπαιδεύουμε όλα τα βάρη
resnetf.trainable = True
# Επιλέγουμε την έξοδο του τελευταίου συνελικτικού επιπέδου. 
# Θα μπορούσαμε να επιλέξουμε οποιοδήποτε άλλο επίπεδο.
conv_layer = resnetf.get_layer('conv5_block3_out').output
# Όπως προτείνεται και στο ResNet paper, επιλέγουμε, για κάθε εικόνα εξόδου 
# να πάρουμε το μέσο όρο (και όχι να χρησιμοποιήσουμε τη Flatten())
x = tf.keras.layers.GlobalAveragePooling2D()(conv_layer)
# Προσθέτουμε τα ταλευταία επίπεδα κατηγοριοποίησης
x = tf.keras.layers.Dense(3, activation='softmax')(x)
# Ορίζουμε το ενδιάμεσο μοντέλο
modelf = tf.keras.Model(resnetf.input, [x])

# Και ορίζουμε το συνολικό μοντέλο, όπως προηγουμένως
input = tf.keras.layers.Input((224,224,3))
pre = tf.keras.layers.Resizing(224, 224)(input)
out = modelf(pre)
model2 = tf.keras.Model(input, outputs=[out])

In [17]:
model2.compile(loss='categorical_crossentropy'
              , optimizer = keras.optimizers.Adam(),metrics=["accuracy"])

model2.fit(ds_train, validation_data=ds_val,  batch_size=128, epochs=20)

score = model2.evaluate(ds_test)
print(score)

Epoch 1/20
131/131 [==============================] - 1590s 12s/step - loss: 0.7460 - accuracy: 0.6611 - val_loss: 4.3704 - val_accuracy: 0.2570
Epoch 2/20
131/131 [==============================] - 1298s 10s/step - loss: 0.5742 - accuracy: 0.7100 - val_loss: 4.2974 - val_accuracy: 0.2570
Epoch 3/20
131/131 [==============================] - 1377s 11s/step - loss: 0.5479 - accuracy: 0.7290 - val_loss: 3.4042 - val_accuracy: 0.2570
Epoch 4/20
131/131 [==============================] - 1643s 13s/step - loss: 0.5403 - accuracy: 0.7378 - val_loss: 2.3152 - val_accuracy: 0.4717
Epoch 5/20
131/131 [==============================] - 1595s 12s/step - loss: 0.5283 - accuracy: 0.7347 - val_loss: 4.7001 - val_accuracy: 0.2570
Epoch 6/20
131/131 [==============================] - 1481s 11s/step - loss: 0.5316 - accuracy: 0.7378 - val_loss: 2.6293 - val_accuracy: 0.2665
Epoch 7/20
131/131 [==============================] - 1587s 12s/step - loss: 0.5141 - accuracy: 0.7534 - val_loss: 4.8837 - val_ac

In [18]:
def get_model():    
    #Input shape = [width, height, color channels]
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    hidden=inputs
    
    # Block One
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    # Block Two
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    # Block Three
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    # Block four
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    # Head
    hidden = layers.Flatten()(hidden)
    hidden = layers.Dense(64, activation='relu',kernel_regularizer=regularizers.L2(0.001))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    #Final Layer (Output)
    output = layers.Dense(3, activation='softmax')(hidden)     
    model = keras.Model(inputs,output)    
    return model

In [19]:
from tensorflow.keras import layers
from tensorflow.keras import regularizers

keras.backend.clear_session()

model = get_model()
model.compile(loss='categorical_crossentropy'
              , optimizer = keras.optimizers.Adam(), metrics='accuracy')

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        36928 

In [20]:
history = model.fit(ds_train,
          batch_size = BATCH, 
          epochs = 50,
          validation_data=ds_val,
          callbacks=early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 459s 3s/step - loss: 1.0282 - accuracy: 0.5279 - val_loss: 0.8039 - val_accuracy: 0.6472
Epoch 2/50
130/130 [==============================] - 493s 4s/step - loss: 0.8019 - accuracy: 0.6287 - val_loss: 0.6919 - val_accuracy: 0.6951
Epoch 3/50
130/130 [==============================] - 476s 4s/step - loss: 0.7399 - accuracy: 0.6599 - val_loss: 0.6600 - val_accuracy: 0.6942
Epoch 4/50
130/130 [==============================] - 483s 4s/step - loss: 0.7136 - accuracy: 0.6716 - val_loss: 0.6288 - val_accuracy: 0.7152
Epoch 5/50
130/130 [==============================] - 486s 4s/step - loss: 0.6951 - accuracy: 0.6781 - val_loss: 0.6364 - val_accuracy: 0.7018
Epoch 6/50
130/130 [==============================] - 484s 4s/step - loss: 0.6963 - accuracy: 0.6750 - val_loss: 0.6301 - val_accuracy: 0.7057
Epoch 7/50
130/130 [==============================] - 446s 3s/step - loss: 0.6820 - accuracy: 0.6819 - val_loss: 0.6069 - val_accuracy: 0.7124

In [21]:
def showScore():
    score = model.evaluate(ds_test, steps = len(val_df)/BATCH, verbose = 0)
    print('Val loss:', score[0])
    print('Val accuracy:', score[1])
    best_score = max(history.history['accuracy'])
    print('Best score:',best_score)
    

In [22]:
showScore()

Val loss: 1.3443892002105713
Val accuracy: 0.4848484992980957
Best score: 0.704005777835846


In [24]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min',
)

In [26]:
history = model.fit(ds_train,
          batch_size = BATCH, 
          epochs = 50,
          validation_data=ds_val,
          callbacks=early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 229s 2s/step - loss: 0.6269 - accuracy: 0.7033 - val_loss: 0.6075 - val_accuracy: 0.7095
Epoch 2/50
130/130 [==============================] - 249s 2s/step - loss: 0.6282 - accuracy: 0.7069 - val_loss: 0.5652 - val_accuracy: 0.7277
Epoch 3/50
130/130 [==============================] - 268s 2s/step - loss: 0.6165 - accuracy: 0.7088 - val_loss: 0.5777 - val_accuracy: 0.7325
Epoch 4/50
130/130 [==============================] - 253s 2s/step - loss: 0.6271 - accuracy: 0.7083 - val_loss: 0.5582 - val_accuracy: 0.7267
Epoch 5/50
130/130 [==============================] - 191s 1s/step - loss: 0.6157 - accuracy: 0.7119 - val_loss: 0.5743 - val_accuracy: 0.7267
Epoch 6/50
130/130 [==============================] - 190s 1s/step - loss: 0.6021 - accuracy: 0.7186 - val_loss: 0.5902 - val_accuracy: 0.7220
Epoch 7/50
130/130 [==============================] - 190s 1s/step - loss: 0.6061 - accuracy: 0.7165 - val_loss: 0.5728 - val_accuracy: 0.7258

In [27]:
showScore()

Val loss: 0.5279433131217957
Val accuracy: 0.939393937587738
Best score: 0.732309877872467


In [28]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=4,
    mode='min',
)

In [29]:
history = model.fit(ds_train,
          batch_size = 200, 
          epochs = 50,
          validation_data=ds_val,
          callbacks=early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 211s 2s/step - loss: 0.5840 - accuracy: 0.7282 - val_loss: 0.5526 - val_accuracy: 0.7392
Epoch 2/50
130/130 [==============================] - 284s 2s/step - loss: 0.5756 - accuracy: 0.7268 - val_loss: 0.5477 - val_accuracy: 0.7450
Epoch 3/50
130/130 [==============================] - 277s 2s/step - loss: 0.5859 - accuracy: 0.7297 - val_loss: 0.5585 - val_accuracy: 0.7335
Epoch 4/50
130/130 [==============================] - 305s 2s/step - loss: 0.5784 - accuracy: 0.7311 - val_loss: 0.5600 - val_accuracy: 0.7229
Epoch 5/50
130/130 [==============================] - 304s 2s/step - loss: 0.5805 - accuracy: 0.7354 - val_loss: 0.5514 - val_accuracy: 0.7459
Epoch 6/50
130/130 [==============================] - 254s 2s/step - loss: 0.5739 - accuracy: 0.7335 - val_loss: 0.6115 - val_accuracy: 0.7028


In [30]:
showScore()

Val loss: 0.5679208040237427
Val accuracy: 0.9090909361839294
Best score: 0.7354281544685364


In [31]:
print(len(train_df)/BATCH)
print(len(val_df)/BATCH)

130.28125
32.59375


In [32]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=8,
    mode='min',
)


In [33]:
history = model.fit(ds_train,
          batch_size = 150, 
          epochs = 50,
          validation_data=ds_val,
          callbacks=early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 276s 2s/step - loss: 0.5686 - accuracy: 0.7385 - val_loss: 0.5653 - val_accuracy: 0.7383
Epoch 2/50
130/130 [==============================] - 217s 2s/step - loss: 0.5658 - accuracy: 0.7361 - val_loss: 0.5669 - val_accuracy: 0.7267
Epoch 3/50
130/130 [==============================] - 210s 2s/step - loss: 0.5679 - accuracy: 0.7381 - val_loss: 0.5780 - val_accuracy: 0.7162
Epoch 4/50
130/130 [==============================] - 205s 2s/step - loss: 0.5721 - accuracy: 0.7335 - val_loss: 0.5651 - val_accuracy: 0.7325
Epoch 5/50
130/130 [==============================] - 222s 2s/step - loss: 0.5660 - accuracy: 0.7345 - val_loss: 0.5597 - val_accuracy: 0.7277
Epoch 6/50
130/130 [==============================] - 216s 2s/step - loss: 0.5691 - accuracy: 0.7390 - val_loss: 0.5521 - val_accuracy: 0.7354
Epoch 7/50
130/130 [==============================] - 215s 2s/step - loss: 0.5583 - accuracy: 0.7424 - val_loss: 0.5681 - val_accuracy: 0.7277

In [34]:
showScore()

Val loss: 0.48073261976242065
Val accuracy: 0.8787878751754761
Best score: 0.7481410503387451


In [35]:
def get_model():    
    #Input shape = [width, height, color channels]
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    hidden=inputs
    
    # Block One
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    # Block Two
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    # Block Three
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    # Block four
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    # Head
    hidden = layers.Flatten()(hidden)
    hidden = layers.Dense(64, activation='relu',kernel_regularizer=regularizers.L2(0.01))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    #Final Layer (Output)
    output = layers.Dense(3, activation='softmax')(hidden)     
    model = keras.Model(inputs,output)    
    return model

In [36]:
history = model.fit(ds_train,
          batch_size = 150, 
          epochs = 50,
          validation_data=ds_val,
          callbacks=early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 240s 2s/step - loss: 0.5495 - accuracy: 0.7568 - val_loss: 0.5671 - val_accuracy: 0.7392
Epoch 2/50
130/130 [==============================] - 254s 2s/step - loss: 0.5547 - accuracy: 0.7479 - val_loss: 0.5702 - val_accuracy: 0.7315
Epoch 3/50
130/130 [==============================] - 250s 2s/step - loss: 0.5413 - accuracy: 0.7501 - val_loss: 0.5895 - val_accuracy: 0.7152
Epoch 4/50
130/130 [==============================] - 237s 2s/step - loss: 0.5500 - accuracy: 0.7462 - val_loss: 0.5799 - val_accuracy: 0.7363
Epoch 5/50
130/130 [==============================] - 250s 2s/step - loss: 0.5583 - accuracy: 0.7453 - val_loss: 0.5530 - val_accuracy: 0.7335
Epoch 6/50
130/130 [==============================] - 225s 2s/step - loss: 0.5456 - accuracy: 0.7486 - val_loss: 0.5754 - val_accuracy: 0.7373
Epoch 7/50
130/130 [==============================] - 238s 2s/step - loss: 0.5400 - accuracy: 0.7503 - val_loss: 0.5704 - val_accuracy: 0.7373

In [37]:
showScore()

Val loss: 1.1652361154556274
Val accuracy: 0.6363636255264282
Best score: 0.7570160627365112


In [40]:
def get_model():    
    #Input shape = [width, height, color channels]
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    hidden=inputs
    
    # Block One
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    # Block Two
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    # Block Three
    hidden = layers.Conv2D(128, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    # Block four
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    # Head
    hidden = layers.Flatten()(hidden)
    hidden = layers.Dense(32, activation='relu',kernel_regularizer=regularizers.L2(0.001))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    #Final Layer (Output)
    output = layers.Dense(3, activation='softmax')(hidden)     
    model = keras.Model(inputs,output)    
    return model



In [41]:
model = get_model()
model.compile(loss='categorical_crossentropy'
              , optimizer = keras.optimizers.Adam(), metrics='accuracy')

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_11 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 52, 52, 128)       7385

In [42]:
history = model.fit(ds_train,
          batch_size = 140, 
          epochs = 50,
          validation_data=ds_val,
          callbacks = early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 275s 2s/step - loss: 1.0908 - accuracy: 0.4773 - val_loss: 1.0731 - val_accuracy: 0.4880
Epoch 2/50
130/130 [==============================] - 248s 2s/step - loss: 0.9175 - accuracy: 0.5757 - val_loss: 0.7036 - val_accuracy: 0.6903
Epoch 3/50
130/130 [==============================] - 246s 2s/step - loss: 0.7498 - accuracy: 0.6464 - val_loss: 0.7806 - val_accuracy: 0.6414
Epoch 4/50
130/130 [==============================] - 232s 2s/step - loss: 0.7181 - accuracy: 0.6661 - val_loss: 0.6544 - val_accuracy: 0.6865
Epoch 5/50
130/130 [==============================] - 231s 2s/step - loss: 0.6938 - accuracy: 0.6726 - val_loss: 0.6078 - val_accuracy: 0.7085
Epoch 6/50
130/130 [==============================] - 231s 2s/step - loss: 0.6873 - accuracy: 0.6740 - val_loss: 0.6255 - val_accuracy: 0.7028
Epoch 7/50
130/130 [==============================] - 230s 2s/step - loss: 0.6610 - accuracy: 0.6800 - val_loss: 0.6043 - val_accuracy: 0.7124

In [46]:

print(ds_test[1])

(array([[[[0.16078432, 0.16078432, 0.16078432],
         [0.13725491, 0.13725491, 0.13725491],
         [0.11764707, 0.11764707, 0.11764707],
         ...,
         [0.43529415, 0.43529415, 0.43529415],
         [0.47058827, 0.47058827, 0.47058827],
         [0.40784317, 0.40784317, 0.40784317]],

        [[0.15686275, 0.15686275, 0.15686275],
         [0.1254902 , 0.1254902 , 0.1254902 ],
         [0.12156864, 0.12156864, 0.12156864],
         ...,
         [0.4784314 , 0.4784314 , 0.4784314 ],
         [0.44705886, 0.44705886, 0.44705886],
         [0.42352945, 0.42352945, 0.42352945]],

        [[0.15294118, 0.15294118, 0.15294118],
         [0.14901961, 0.14901961, 0.14901961],
         [0.13725491, 0.13725491, 0.13725491],
         ...,
         [0.4431373 , 0.4431373 , 0.4431373 ],
         [0.43137258, 0.43137258, 0.43137258],
         [0.4156863 , 0.4156863 , 0.4156863 ]],

        ...,

        [[0.15686275, 0.15686275, 0.15686275],
         [0.16078432, 0.16078432, 0.16078432

In [47]:
showScore()

Val loss: 0.8908036947250366
Val accuracy: 0.6666666865348816
Best score: 0.7313504219055176


In [48]:
def get_model():    
    #Input shape = [width, height, color channels]
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    hidden=inputs
    
    
    hidden = layers.Conv2D(16, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    hidden = layers.Dropout(0.2)(hidden)
      
    
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)    
    
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Flatten()(hidden)
    hidden = layers.Dense(16, activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    
    output = layers.Dense(3, activation='softmax')(hidden)     
    model = keras.Model(inputs,output)    
    return model

In [49]:
model = get_model()
model.compile(loss='categorical_crossentropy'
              , optimizer = keras.optimizers.Adam(), metrics='accuracy')

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_16 (Conv2D)          (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 111, 111, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 54, 54, 32)       0         
 g2D)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 54, 54, 32)        0   

In [50]:
history = model.fit(ds_train,
          batch_size = 128, 
          epochs = 50,
          validation_data=ds_val,
          callbacks=early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 203s 2s/step - loss: 1.0645 - accuracy: 0.4812 - val_loss: 1.0559 - val_accuracy: 0.4851
Epoch 2/50
130/130 [==============================] - 212s 2s/step - loss: 1.0420 - accuracy: 0.4855 - val_loss: 1.0008 - val_accuracy: 0.4851
Epoch 3/50
130/130 [==============================] - 218s 2s/step - loss: 0.9801 - accuracy: 0.5018 - val_loss: 0.9252 - val_accuracy: 0.5791
Epoch 4/50
130/130 [==============================] - 231s 2s/step - loss: 0.9430 - accuracy: 0.5419 - val_loss: 0.9039 - val_accuracy: 0.5916
Epoch 5/50
130/130 [==============================] - 218s 2s/step - loss: 0.9252 - accuracy: 0.5536 - val_loss: 0.8978 - val_accuracy: 0.5839
Epoch 6/50
130/130 [==============================] - 230s 2s/step - loss: 0.8153 - accuracy: 0.6268 - val_loss: 0.7783 - val_accuracy: 0.6174
Epoch 7/50
130/130 [==============================] - 223s 2s/step - loss: 0.7463 - accuracy: 0.6536 - val_loss: 0.6457 - val_accuracy: 0.7028

In [51]:
showScore()

Val loss: 0.7353569269180298
Val accuracy: 0.7272727489471436
Best score: 0.751978874206543
